<a href="https://colab.research.google.com/github/tanishjanani/MobileNet-SSD/blob/master/tanish_startup_data_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
warnings.filterwarnings('ignore')

In [ ]:
company = pd.read_csv("companies.csv")


In [ ]:
company.head()
shape = company.shape

# Print the shape
print(f"Shape of the DataFrame: {shape}")


Shape of the DataFrame: (2798, 20)


In [ ]:
# Delete specified columns
columns_to_delete = [
    'region', 'city', 'state_code',  # Granularity
    'id', 'Unnamed: 0.1', 'entity_type', 'entity_id', 'parent_id', 'created_by',
    'created_at', 'updated_at',  # Redundant
    'domain', 'homepage_url', 'twitter_username', 'logo_url', 'logo_width', 'logo_height',
    'short_description', 'description', 'overview', 'tag_list', 'name', 'normalized_name', 'permalink',
    'invested_companies'  # Irrelevant
]
company = company.drop(columns=columns_to_delete, errors='ignore')

# Delete duplicate rows
company.drop_duplicates(inplace=True)

# Delete columns with more than 98% null values
threshold = 0.98
company = company.loc[:, company.isnull().mean() < threshold]

In [ ]:
company.head()

,category_code,status,founded_at,closed_at,country_code,first_investment_at,last_investment_at,investment_rounds,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,lat,lng
0,web,operating,2005-10-17,NaN,USA,NaN,NaN,NaN,2005-10-01,2008-05-19,3.0,39750000.0,2010-09-05,2013-09-18,5.0,17.0,47.606209,-122.332071
1,games_video,acquired,NaN,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,34.021122,-118.396467
2,games_video,acquired,NaN,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003-02-01,2011-09-23,4.0,12.0,37.562992,-122.325525
3,network_hosting,operating,2008-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,games_video,operating,2008-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
shape = company.shape

# Print the shape
print(f"Shape of the DataFrame: {shape}")

Shape of the DataFrame: (4891, 18)


In [ ]:
company.dropna(subset=['status', 'country_code', 'category_code', 'founded_at'], inplace=True)

In [ ]:
# b. Delete outliers for 'funding_total_usd' and 'funding_rounds'
# (Using IQR method as an example - adapt to your data and desired method)

for column in ['funding_total_usd', 'funding_rounds']:
    Q1 = company[column].quantile(0.25)
    Q3 = company[column].quantile(0.75)
    IQR = Q3 - Q1
    company = company[~((company[column] < (Q1 - 1.5 * IQR)) | (company[column] > (Q3 + 1.5 * IQR)))]

# **B data transformation**

In [ ]:
# a. Convert date columns to years
date_columns = ['founded_at', 'closed_at', 'first_funding_at', 'last_funding_at', 'first_milestone_at', 'last_milestone_at']
for column in date_columns:
    company[column] = pd.to_datetime(company[column]).dt.year

In [ ]:
# Generalize category_code
category_frequencies = company['category_code'].value_counts(normalize=True)
categories_to_keep = category_frequencies[category_frequencies >= 0.05].index
company['category_code'] = company['category_code'].apply(lambda x: x if x in categories_to_keep else 'Other')

# Generalize country_code
country_frequencies = company['country_code'].value_counts(normalize=True)
countries_to_keep = country_frequencies[country_frequencies >= 0.05].index
company['country_code'] = company['country_code'].apply(lambda x: x if x in countries_to_keep else 'Other')

In [ ]:
company['isClosed'] = company['status'].apply(lambda x: 1 if x == 'closed' else 0)

In [ ]:
company['active_days'] = (company['closed_at'] - company['founded_at']).fillna(0)

In [ ]:
import pandas as pd

# ... (Your existing code)

# Calculate active_days
company['active_days'] = company.apply(lambda row: (row['closed_at'] - row['founded_at'])
                                       if row['status'] == 'closed'
                                       else (pd.Timestamp.now().year - row['founded_at']), axis=1)

# ... (Rest of your code)

In [ ]:
company.head(2111)

,category_code,status,founded_at,closed_at,country_code,first_investment_at,last_investment_at,investment_rounds,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,lat,lng,isClosed,active_days
0,web,operating,1970,NaN,USA,NaN,NaN,NaN,1970.0,1970.0,3.0,39750000.0,1970.0,1970.0,5.0,17.0,47.606209,-122.332071,0,55
5,advertising,operating,1970,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,30.427755,-9.598107,0,55
6,Other,operating,1970,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.307159,73.181219,0,55
12,advertising,operating,1970,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0,1970.0,1.0,2.0,35.686975,-105.937799,0,55
13,web,acquired,1970,NaN,USA,NaN,NaN,NaN,1970.0,1970.0,1.0,5000000.0,1970.0,1970.0,3.0,14.0,37.386052,-122.083851,0,55
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4361,advertising,operating,1970,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,40.448219,-79.924262,0,55
4362,software,operating,1970,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.589045,-79.644120,0,55
4364,consulting,operating,1970,NaN,Other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,-26.204103,28.047305,0,55
4365,Other,operating,1970,NaN,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970.0,1970.0,2.0,5.0,37.774929,-122.419415,0,55


In [ ]:
filtered_company = company[company['founded_at'] > 1995]
filtered_company.head()

,category_code,status,founded_at,closed_at,country_code,first_investment_at,last_investment_at,investment_rounds,first_funding_at,last_funding_at,funding_rounds,funding_total_usd,first_milestone_at,last_milestone_at,milestones,relationships,lat,lng,isClosed,active_days
